##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Data Validation

***Exemplo de um componente chave do TensorFlow Extended (TFX)***

Observação: você pode executar este exemplo agora mesmo num notebook estilo Jupyter, sem necessidade de configuração! Basta clicar em "Executar no Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Este exemplo de notebook colab ilustra como o TensorFlow  Data Validation (TFDV) pode ser usado para investigar e visualizar seu dataset. Isto inclui examinar estatísticas descritivas, inferir um esquema, verificar e corrigir anomalias e verificar deriva e desvios em nosso dataset. É importante compreender as características do seu dataset, incluindo como ele pode mudar ao longo do tempo no seu pipeline de produção. Também é importante procurar anomalias nos seus dados e comparar os seus datasets de treinamento, avaliação e serviço para garantir que são consistentes.

Usaremos o [dataset Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) disponibilizado pela cidade de Chicago.

Observação: Este site fornece aplicativos que utilizam dados que foram modificados para uso a partir de sua fonte original obtida em www.cityofchicago.org, site oficial da cidade de Chicago. A cidade de Chicago não faz nenhuma reivindicação quanto ao conteúdo, precisão, atualidade ou integridade de qualquer um dos dados fornecidos neste site. Os dados fornecidos neste site estão sujeitos a alterações a qualquer momento. Entende-se que os dados fornecidos neste site são utilizados por sua conta e risco.

[Leia mais](https://cloud.google.com/bigquery/public-data/chicago-taxi) sobre o dataset no [Google BigQuery](https://cloud.google.com/bigquery/). Explore o dataset completo no [BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips).

Ponto-chave: como modelador e desenvolvedor, pense em como esses dados são usados ​​e nos possíveis benefícios e danos que as previsões de um modelo podem causar. Um modelo como este poderia reforçar um viés social e disparidades. Uma determinada característica é relevante para o problema que você quer resolver ou será que ela vai introduzir um viés? Para mais informações, leia sobre [Equidade em ML](https://developers.google.com/machine-learning/fairness-overview/).

As colunas do dataset são:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## Instale e importe os pacotes

Instale os pacotes para o TensorFlow Data Validation.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale os pacotes do Data Validation

Instale os pacotes e dependências do TensorFlow Data Validation, o que leva alguns minutos. Você poderá ver avisos e erros relacionados a versões de dependências incompatíveis, que serão resolvidos na próxima seção.

In [ ]:
print('Installing TensorFlow Data Validation')
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'

### Importe o TensorFlow e recarregue os pacotes atualizados

A etapa anterior atualiza os pacotes padrão no ambiente Google Colab, portanto, você deve recarregar os recursos do pacote para resolver as novas dependências.

Observação: Esta etapa resolve o erro de dependência da instalação. Se você ainda estiver enfrentando problemas de execução de código depois de executar esse código, reinicie o runtime (Runtime &gt; Restart runtime...).

In [ ]:
import pkg_resources
import importlib
importlib.reload(pkg_resources)

Confira as versões do TensorFlow e do Data Validation antes de continuar. 

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

## Carregue o dataset

Baixaremos nosso dataset do Google Cloud Storage.

In [ ]:
import os
import tempfile, urllib, zipfile

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R {os.path.join(BASE_DIR, 'data')}

## Compute e visualize estatísticas

Primeiro usaremos o [`tfdv.generate_statistics_from_csv`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv) para computar estatísticas para nossos dados de treinamento. (ignore os avisos rápidos)

O TFDV pode computar [estatísticas](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto) descritivas que fornecem uma visão geral rápida dos dados em termos das características presentes e das formas de suas distribuições de valor.

Internamente, o TFDV usa o framework de processamento paralelo de dados do [Apache Beam](https://beam.apache.org/) para dimensionar a computação de estatísticas em grandes datasets. Para aplicações que desejam uma integração mais profunda com TFDV (por exemplo, anexar geração de estatísticas no final de um pipeline de geração de dados), a API também expõe um Beam PTransform para geração de estatísticas.

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

Agora vamos usar o [`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics), que usa [Facets](https://pair-code.github.io/facets/) para criar uma visualização sucinta dos nossos dados de treinamento:

- Observe que as características numéricas e as características categóricas são visualizadas separadamente e que os gráficos são exibidos mostrando as distribuições de cada característica.
- Observe que as características com valores ausentes ou zero exibem uma porcentagem em vermelho como um indicador visual de que pode haver problemas com exemplos nessas características. A porcentagem é a porcentagem de exemplos que possuem valores ausentes ou nulos para essa características.
- Observe que não há exemplos com valores para `pickup_census_tract`. Esta é uma oportunidade para redução de dimensionalidade!
- Experimente clicar em "expandir" acima dos gráficos para alterar a exibição
- Experimente passar o mouse sobre as barras nos gráficos para exibir contagens e intervalos de buckets
- Experimente alternar entre as escalas logarítmica e linear e observe como a escala logarítmica revela muito mais detalhes sobre a característica categórica `payment_type`
- Experimente selecionar "quantiles" no menu "Chart to show" e passe o mouse sobre os marcadores para mostrar as porcentagens de quantis

In [ ]:
# docs-infra: no-execute
tfdv.visualize_statistics(train_stats)

<!-- <img class="tfo-display-only-on-site" src="images/statistics.png"/> -->

## Infira um esquema

Agora vamos usar [`tfdv.infer_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema) para criar um esquema para nossos dados. Um esquema define restrições para os dados que são relevantes para ML. Exemplos de restrições incluem o tipo de dados de cada característica, seja numérico ou categórico, ou a frequência de sua presença nos dados. Para características categóricas, o esquema também define o domínio – a lista de valores aceitáveis. Como escrever um esquema pode ser uma tarefa chata, especialmente para datasets com muitas características, o TFDV fornece um método para gerar uma versão inicial do esquema com base nas estatísticas descritivas.

Acertar o esquema é importante porque o resto do nosso pipeline de produção dependerá do esquema que o TFDV gera para estar correto. O esquema também fornece documentação para os dados e, portanto, é útil quando diferentes desenvolvedores trabalham nos mesmos dados. Vamos usar [`tfdv.display_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema) para exibir o esquema inferido para que possamos revisá-lo.

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## Confira os dados de avaliação em busca de erros

Até agora, analisamos apenas os dados de treinamento. É importante que nossos dados de avaliação sejam consistentes com nossos dados de treinamento, inclusive que usem o mesmo esquema. Também é importante que os dados de avaliação incluam exemplos de aproximadamente os mesmos intervalos de valores para nossas características numéricas que nossos dados de treinamento, para que nossa cobertura da superfície de perda durante a avaliação seja aproximadamente a mesma que durante o treinamento. O mesmo se aplica a características categóricas. Caso contrário, poderemos ter problemas de treinamento que não serão identificados durante a avaliação, pois não avaliamos parte da nossa superfície de perdas.

- Observe que cada característica agora inclui estatísticas para os datasets de treinamento e avaliação.
- Observe que os gráficos agora têm os datasets de treinamento e avaliação sobrepostos, facilitando sua comparação.
- Observe que os gráficos agora incluem uma visualização de porcentagens, que pode ser combinada com o log ou com as escalas lineares padrão.
- Observe que a média e a mediana de `trip_miles` são diferentes para os datasets de treinamento e de avaliação. Isto causará problemas?
- Uau, as `tips` (gorjetas) máximas são muito diferentes para os datasets de treinamento e de avaliação. Isto causará problemas?
- Clique em expandir no gráfico Numeric Features e selecione a escala logarítmica. Revise a característica `trip_seconds` e observe a diferença no máximo. A avaliação perderá partes da superfície de perda?

In [ ]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

In [ ]:
# docs-infra: no-execute
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

<!-- <img class="tfo-display-only-on-site" src="images/statistics_eval.png"/> -->

## Verifique se há anomalias de avaliação

Será que nosso dataset de avaliação corresponde ao esquema do nosso dataset de treinamento? Isto é especialmente importante para características categóricas, onde queremos identificar o intervalo de valores aceitáveis.

Ponto-chave: O que aconteceria se tentássemos avaliar usando dados com valores de características categóricas que não estivessem em nosso dataset de treinamento? E quanto às características numéricas que estão fora dos intervalos em nosso dataset de treinamento?

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## Corrija anomalias de avaliação no esquema

Ops! Parece que temos alguns valores novos para `company` em nossos dados de avaliação, que não tínhamos em nossos dados de treinamento. Também temos um novo valor para `payment_type`. Estas devem ser consideradas anomalias, mas o que decidimos fazer sobre elas depende do nosso conhecimento sobre o domínio dos dados. Se uma anomalia realmente indicar um erro de dados, os dados subjacentes deverão ser corrigidos. Caso contrário, podemos simplesmente atualizar o esquema para incluir os valores no dataset de avaliação.

Ponto-chave: Como os resultados da nossa avaliação seriam afetados se não corrigíssemos esses problemas?

A menos que alteremos nosso dataset de avaliação, não poderemos corrigir tudo, mas podemos corrigir coisas no esquema que nos sentimos confortáveis ​​em aceitar. Isto inclui relaxar nossa visão do que é e do que não é uma anomalia para características específicas, bem como atualizar nosso esquema para incluir valores ausentes para características categóricas. O TFDV nos permitiu descobrir o que precisamos corrigir.

Vamos fazer essas correções agora e depois revisar mais uma vez.

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

Ei, olha só isso! Verificamos que os dados de treinamento e avaliação agora são consistentes! Obrigado TFDV ;)

## Ambientes de esquema

Também dividimos um dataset de 'serviço' para este exemplo, portanto, devemos verificar isso também. Por padrão, todos os datasets de um pipeline devem usar o mesmo esquema, mas muitas vezes há exceções. Por exemplo, na aprendizagem supervisionada precisamos incluir rótulos em nosso dataset, mas quando servimos o modelo para inferência os rótulos não serão incluídos. Em alguns casos, é necessária a introdução de pequenas variações de esquema.

**Ambientes** (environments) podem ser usados ​​para expressar tais requisitos. Em particular, as características do esquema podem ser associadas a um conjunto de ambientes usando `default_environment`, `in_environment` e `not_in_environment`.

Por exemplo, neste dataset, a característica `tips` está incluída como rótulo para treinamento, mas está faltando nos dados de veiculação. Sem o ambiente especificado, ela aparecerá como uma anomalia.

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Cuidaremos da característica `tips` abaixo. Também temos um valor INT em nossos segundos de viagem, onde nosso esquema esperava um FLOAT. Ao nos conscientizar dessa diferença, o TFDV ajuda a descobrir inconsistências na forma como os dados são gerados para treinamento e serviço. É muito fácil não ter consciência de problemas como esse até que o desempenho do modelo seja prejudicado, às vezes de forma catastrófica. Pode ou não ser um problema significativo, mas em qualquer caso, deve ser motivo para uma investigação mais aprofundada.

Neste caso, podemos converter com segurança valores INT em FLOATs, por isso queremos dizer ao TFDV para usar nosso esquema para inferir o tipo. Vamos fazer isso agora.

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Agora só temos a característica `tips` (que é o nosso rótulo) aparecendo como uma anomalia ('Coluna descartada'). É claro que não esperamos ter rótulos em nossos dados de serviço, então vamos dizer ao TFDV para ignorar isso.

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## Verifique se há deriva e desvio

Além de verificar se um dataset está em conformidade com as expectativas definidas no esquema, o TFDV também fornece funcionalidades para detectar derivas e desvios. O TFDV realiza essa verificação comparando as estatísticas dos diferentes datasets com base nos comparadores de deriva/desvio especificados no esquema.

### Drift (deriva)

A detecção de deriva (drift) é suportada para características categóricas e entre spans consecutivos de dados (ou seja, entre o span N e o span N+1), como entre diferentes dias de dados de treinamento. Expressamos a deriva em termos de [distância L-infinito](https://en.wikipedia.org/wiki/Chebyshev_distance) e você pode definir a distância limite para receber avisos quando a deriva for maior do que o aceitável. Definir a distância correta normalmente é um processo iterativo que requer conhecimento do domínio e experimentação.

### Skew (desvio)

O TFDV pode detectar três tipos diferentes de desvio em seus dados: desvio de esquema, desvio de característica e desvio de distribuição.

#### Desvio de esquema

O desvio de esquema ocorre quando os dados de treinamento e entrega não estão em conformidade com o mesmo esquema. Espera-se que os dados de treinamento e de entrega sigam o mesmo esquema. Quaisquer desvios esperados entre os dois (como a característica de rótulo estar presente apenas nos dados de treinamento, mas não no serviço) devem ser especificados por meio do campo de ambientes (environments) no esquema.

#### Desvio de características

O desvio de características ocorre quando os valores de características nos quais um modelo é treinado são diferentes dos valores de características que ele vê no momento do serviço. Por exemplo, isto pode acontecer quando:

- Uma fonte de dados que fornece alguns valores de características é modificada entre o tempo de treinamento e o tempo de serviço
- Existe uma lógica diferente para gerar características entre treinar e servir. Por exemplo, se você aplicar alguma transformação apenas em um dos dois caminhos de código.

#### Desvio de distribuição

O desvio de distribuição ocorre quando a distribuição do dataset de treinamento é significativamente diferente da distribuição do dataset de serviço. Uma das principais causas do desvio de distribuição é o uso de código ou fontes de dados diferentes para gerar o dataset de treinamento. Outro motivo é um mecanismo de amostragem defeituoso que escolhe uma subamostra não representativa dos dados de serviço para treinar.

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

Neste exemplo, vemos alguma deriva, mas está bem abaixo do limite que definimos.

## Congele o esquema

Agora que o esquema foi revisado e selecionado, iremos armazená-lo num arquivo para refletir seu estado “congelado”.

In [ ]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

## Quando usar o TFDV

É fácil achar que o TFDV se aplica apenas ao início do seu pipeline de treinamento, como fizemos aqui, mas na verdade ele tem muitos usos. Aqui estão mais alguns:

- Validar novos dados para inferência para garantir que não começamos repentinamente a receber características ruins
- Validar novos dados para inferência para garantir que nosso modelo foi treinado naquela parte da superfície de decisão
- Validar nossos dados depois de transformá-los e fazer engenharia de características (provavelmente usando [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started)) para garantir que não fizemos algo errado